---
jupyter: Python3
date: 2022-10-29
title: Bigquery_Array, Struct 조합 사용과 Cartesian Product
toc: true
categories: Bigquery
author: limyj0708
comments:
  giscus: 
    repo: quarto-dev/quarto-docs
format:
    html:
        page-layout: full
---

# Bigquery_Array, Struct 조합 사용과 Cartesian Product

- 이런 일을 한 이유
  - 지역 던전 별, 이탈 유무 별, 누적과금대 별 지역 던전 클리어 유저 수를 구해야 함
  - 던전 로그에서 지역 던전 도달 유저 수를 그냥 구하면 도달하지 못한 지역 던전은 아예 출력이 되질 않음. 도달 로그 자체가 없을 것이므로.
     - 해당 유저 세그먼트에서 아무도 도달하지 못한 지역 던전이 있으면, 테이블이 쥐가 파먹은 것 처럼 중간이 비게 된다.
     - 아무도 도달하지 못한 지역 던전은 값이 0으로 뜨게 하고 싶다.
  - 그래서 생각한 것이, 미리 모든 카테고리를 곱집합(cartesian product)한 결과물을 기반 테이블로 만들어 두고, 지역 던전 도달 유저 수를 LEFT JOIN으로 기반 테이블에 붙이는 방법임.

```SQL
DECLARE leaving_check ARRAY<INT64>;
DECLARE acc_sales ARRAY<STRING>;
DECLARE region_quest ARRAY<STRUCT<map_key INT64, map_name STRING, quest_act INT64, quest_chapter INT64, quest_id INT64, quest_name STRING>>;
-- STRUCT 정의 할 때 Alias를 붙여주지 않으면 supertype 에러가 발생함

SET leaving_check = [0,1];
SET acc_sales = [
  '1. 무과금 (0원)'
  ,'2. 베이직 (1200원)'
  ,'3. 소과금' 
  ,'4. 중과금'
  ,'5. 중고과금'
  ,'6. 고과금'
  ,'7. VIP'
  ,'8. VVIP'
  ,'9. 고래유저'
];
SET region_quest = [ -- SET에서는 alias를 붙여주지 않아도 되지만, 붙여주면 쿼리 속도가 훨씬 빨라졌음.
    STRUCT(dummy_key_1111 AS map_key, "dummy_name_1111" AS map_name, 1 AS quest_act, 13 AS quest_chapter, dummy_id_1111 AS quest_id, "dummy_q_name_1111" AS quest_name)
    ,STRUCT(dummy_key_1112 AS map_key, "dummy_name_1112" AS map_name, 1 AS quest_act, 24 AS quest_chapter, dummy_id_1112 AS quest_id, "dummy_q_name_1112" AS quest_name)
    ,STRUCT(dummy_key_1113 AS map_key, "dummy_name_1113" AS map_name, 1 AS quest_act, 29 AS quest_chapter, dummy_id_1113 AS quest_id, "dummy_q_name_1113" AS quest_name)
    ,STRUCT(dummy_key_1114 AS map_key, "dummy_name_1114" AS map_name, 1 AS quest_act, 37 AS quest_chapter, dummy_id_1114 AS quest_id, "dummy_q_name_1114" AS quest_name)
    ,STRUCT(dummy_key_1115 AS map_key, "dummy_name_1115" AS map_name, 1 AS quest_act, 33 AS quest_chapter, dummy_id_1115 AS quest_id, "dummy_q_name_1115" AS quest_name)
    ,STRUCT(dummy_key_1116 AS map_key, "dummy_name_1116" AS map_name, 2 AS quest_act, 5 AS quest_chapter, dummy_id_1116 AS quest_id, "dummy_q_name_1116" AS quest_name)
    ,STRUCT(dummy_key_1117 AS map_key, "dummy_name_1117" AS map_name, 2 AS quest_act, 11 AS quest_chapter, dummy_id_1117 AS quest_id, "dummy_q_name_1117" AS quest_name)
    ,STRUCT(dummy_key_1118 AS map_key, "dummy_name_1118" AS map_name, 2 AS quest_act, 16 AS quest_chapter, dummy_id_1118 AS quest_id, "dummy_q_name_1118" AS quest_name)
    ,STRUCT(dummy_key_1119 AS map_key, "dummy_name_1119" AS map_name, 2 AS quest_act, 20 AS quest_chapter, dummy_id_1119 AS quest_id, "dummy_q_name_1119" AS quest_name)
    ,STRUCT(dummy_key_1120 AS map_key, "dummy_name_1120" AS map_name, 2 AS quest_act, 23 AS quest_chapter, dummy_id_1120 AS quest_id, "dummy_q_name_1120" AS quest_name)
    ,STRUCT(dummy_key_1121 AS map_key, "dummy_name_1121" AS map_name, 3 AS quest_act, 20 AS quest_chapter, dummy_id_1121 AS quest_id, "dummy_q_name_1121" AS quest_name)
    ,STRUCT(dummy_key_1122 AS map_key, "dummy_name_1122" AS map_name, 3 AS quest_act, 25 AS quest_chapter, dummy_id_1122 AS quest_id, "dummy_q_name_1122" AS quest_name)
    ,STRUCT(dummy_key_1123 AS map_key, "dummy_name_1123" AS map_name, 3 AS quest_act, 5 AS quest_chapter, dummy_id_1123 AS quest_id, "dummy_q_name_1123" AS quest_name)
    ,STRUCT(dummy_key_1124 AS map_key, "dummy_name_1124" AS map_name, 3 AS quest_act, 10 AS quest_chapter, dummy_id_1124 AS quest_id, "dummy_q_name_1124" AS quest_name)
    ,STRUCT(dummy_key_1125 AS map_key, "dummy_name_1125" AS map_name, 3 AS quest_act, 29 AS quest_chapter, dummy_id_1125 AS quest_id, "dummy_q_name_1125" AS quest_name)
    ,STRUCT(dummy_key_1126 AS map_key, "dummy_name_1126" AS map_name, 3 AS quest_act, 32 AS quest_chapter, dummy_id_1126 AS quest_id, "dummy_q_name_1126" AS quest_name)
    ,STRUCT(dummy_key_1127 AS map_key, "dummy_name_1127" AS map_name, 3 AS quest_act, 35 AS quest_chapter, dummy_id_1127 AS quest_id, "dummy_q_name_1127" AS quest_name)
    ,STRUCT(dummy_key_1128 AS map_key, "dummy_name_1128" AS map_name, 4 AS quest_act, 6 AS quest_chapter, dummy_id_1128 AS quest_id, "dummy_q_name_1128" AS quest_name)
    ,STRUCT(dummy_key_1129 AS map_key, "dummy_name_1129" AS map_name, 4 AS quest_act, 12 AS quest_chapter, dummy_id_1129 AS quest_id, "dummy_q_name_1129" AS quest_name)
    ,STRUCT(dummy_key_1130 AS map_key, "dummy_name_1130" AS map_name, 4 AS quest_act, 19 AS quest_chapter, dummy_id_1130 AS quest_id, "dummy_q_name_1130" AS quest_name)
    ,STRUCT(dummy_key_1131 AS map_key, "dummy_name_1131" AS map_name, 4 AS quest_act, 24 AS quest_chapter, dummy_id_1131 AS quest_id, "dummy_q_name_1131" AS quest_name)
    ,STRUCT(dummy_key_1132 AS map_key, "dummy_name_1132" AS map_name, 4 AS quest_act, 31 AS quest_chapter, dummy_id_1132 AS quest_id, "dummy_q_name_1132" AS quest_name)
    ,STRUCT(dummy_key_1133 AS map_key, "dummy_name_1133" AS map_name, 4 AS quest_act, 37 AS quest_chapter, dummy_id_1133 AS quest_id, "dummy_q_name_1133" AS quest_name)
    ,STRUCT(dummy_key_1134 AS map_key, "dummy_name_1134" AS map_name, 4 AS quest_act, 42 AS quest_chapter, dummy_id_1134 AS quest_id, "dummy_q_name_1134" AS quest_name)
    ,STRUCT(dummy_key_1135 AS map_key, "dummy_name_1135" AS map_name, 4 AS quest_act, 45 AS quest_chapter, dummy_id_1135 AS quest_id, "dummy_q_name_1135" AS quest_name)
    ,STRUCT(dummy_key_1136 AS map_key, "dummy_name_1136" AS map_name, 4 AS quest_act, 48 AS quest_chapter, dummy_id_1136 AS quest_id, "dummy_q_name_1136" AS quest_name)
    ,STRUCT(dummy_key_1137 AS map_key, "dummy_name_1137" AS map_name, 5 AS quest_act, 6 AS quest_chapter, dummy_id_1137 AS quest_id, "dummy_q_name_1137" AS quest_name)
    ,STRUCT(dummy_key_1138 AS map_key, "dummy_name_1138" AS map_name, 5 AS quest_act, 10 AS quest_chapter, dummy_id_1138 AS quest_id, "dummy_q_name_1138" AS quest_name)
    ,STRUCT(dummy_key_1139 AS map_key, "dummy_name_1139" AS map_name, 5 AS quest_act, 15 AS quest_chapter, dummy_id_1139 AS quest_id, "dummy_q_name_1139" AS quest_name)
    ,STRUCT(dummy_key_1140 AS map_key, "dummy_name_1140" AS map_name, 5 AS quest_act, 19 AS quest_chapter, dummy_id_1140 AS quest_id, "dummy_q_name_1140" AS quest_name)
    ,STRUCT(dummy_key_1141 AS map_key, "dummy_name_1141" AS map_name, 5 AS quest_act, 25 AS quest_chapter, dummy_id_1141 AS quest_id, "dummy_q_name_1141" AS quest_name)
    ,STRUCT(dummy_key_1142 AS map_key, "dummy_name_1142" AS map_name, 5 AS quest_act, 30 AS quest_chapter, dummy_id_1142 AS quest_id, "dummy_q_name_1142" AS quest_name)
    ,STRUCT(dummy_key_1143 AS map_key, "dummy_name_1143" AS map_name, 5 AS quest_act, 35 AS quest_chapter, dummy_id_1143 AS quest_id, "dummy_q_name_1143" AS quest_name)
    ,STRUCT(dummy_key_1144 AS map_key, "dummy_name_1144" AS map_name, 5 AS quest_act, 38 AS quest_chapter, dummy_id_1144 AS quest_id, "dummy_q_name_1144" AS quest_name)
    ,STRUCT(dummy_key_1145 AS map_key, "dummy_name_1145" AS map_name, 5 AS quest_act, 41 AS quest_chapter, dummy_id_1145 AS quest_id, "dummy_q_name_1145" AS quest_name)
  ];

WITH base_df AS ( -- 이탈유저유무, 누적과금액별 지역던전 베이스 테이블
  SELECT map.*, leaving, acc_sales_cate
  FROM UNNEST(region_quest) AS map, UNNEST(leaving_check) AS leaving, UNNEST(acc_sales) AS acc_sales_cate
)

SELECT * FROM base_df
```

- 아래와 같이 합집합 테이블이 출력된다.

| Row | map_key | map_name    | quest_act | quest_chapter | quest_id  | quest_name    | leaving | acc_sales_cate     |
|-----|---------|-------------|-----------|---------------|-----------|---------------|---------|--------------------|
|   1 |  dummy_key_1111 | dummy_name_1111 |         1 |            13 | dummy_id_1111 | dummy_q_name_1111 |       0 | 1. 무과금 (0원)    |
|   2 |  dummy_key_1111 | dummy_name_1111 |         1 |            13 | dummy_id_1111 | dummy_q_name_1111 |       0 | 2. 베이직 (1200원) |
|   3 |  dummy_key_1111 | dummy_name_1111 |         1 |            13 | dummy_id_1111 | dummy_q_name_1111 |       0 | 3. 소과금          |
|   4 |  dummy_key_1111 | dummy_name_1111 |         1 |            13 | dummy_id_1111 | dummy_q_name_1111 |       0 | 4. 중과금          |
|   5 |  dummy_key_1111 | dummy_name_1111 |         1 |            13 | dummy_id_1111 | dummy_q_name_1111 |       0 | 5. 중고과금        |
|   6 |  dummy_key_1111 | dummy_name_1111 |         1 |            13 | dummy_id_1111 | dummy_q_name_1111 |       0 | 6. 고과금          |
|   7 |  dummy_key_1111 | dummy_name_1111 |         1 |            13 | dummy_id_1111 | dummy_q_name_1111 |       0 | 7. VIP             |
|   8 |  dummy_key_1111 | dummy_name_1111 |         1 |            13 | dummy_id_1111 | dummy_q_name_1111 |       0 | 8. VVIP            |
|   9 |  dummy_key_1111 | dummy_name_1111 |         1 |            13 | dummy_id_1111 | dummy_q_name_1111 |       0 | 9. 고래유저        |
|  10 |  dummy_key_1111 | dummy_name_1111 |         1 |            13 | dummy_id_1111 | dummy_q_name_1111 |       1 | 1. 무과금 (0원)    |
| ... | ...     | ...         | ...       | ...           | ...       | ...      | ...     | ...                |

- 잘 join해서 사용하면 된다.

```SQL
SELECT bd.map_key, bd.map_name, bd.quest_act, bd.quest_chapter, bd.quest_id, bd.quest_name, bd.not_leaving
  ,bd.acc_sales_cate, IFNULL(ar.total_cnt,0) AS total_cnt, IFNULL(ar.success_cnt,0) AS success_cnt
FROM base_df bd
LEFT JOIN agg_result ar ON 
  (ar.map_key = bd.map_key)
  AND (ar.last_7_day_login = bd.not_leaving)
  AND (ar.total_sales_cate = bd.acc_sales_cate)
ORDER BY quest_act, quest_chapter
```